# Introduction

In [5]:
# Import required packages
import torch
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py

Open `lotka_volterra_data.h5` file on notebook

In [35]:
with h5py.File('lotka_volterra_data.h5', 'r') as f:
    # Access the full dataset
    trajectories = f['trajectories'][:]
    time_points = f['time'][:]

    # Access a single trajectory
    system_id = 0 # First system

Checikng shape of the dataset, we expect trajectories to be of size $(1000 \times 100 \times 2)$, and time_points of size $(100 \times 1)$

In [39]:
print('Time points shape:',time_points.shape)
print('')
print('Trajectory shape (pray/predator):',trajectories.shape)

Time points shape: (100,)

Trajectory shape (pray/predator): (1000, 100, 2)


In [38]:
time_points

array([  0.        ,   2.02020202,   4.04040404,   6.06060606,
         8.08080808,  10.1010101 ,  12.12121212,  14.14141414,
        16.16161616,  18.18181818,  20.2020202 ,  22.22222222,
        24.24242424,  26.26262626,  28.28282828,  30.3030303 ,
        32.32323232,  34.34343434,  36.36363636,  38.38383838,
        40.4040404 ,  42.42424242,  44.44444444,  46.46464646,
        48.48484848,  50.50505051,  52.52525253,  54.54545455,
        56.56565657,  58.58585859,  60.60606061,  62.62626263,
        64.64646465,  66.66666667,  68.68686869,  70.70707071,
        72.72727273,  74.74747475,  76.76767677,  78.78787879,
        80.80808081,  82.82828283,  84.84848485,  86.86868687,
        88.88888889,  90.90909091,  92.92929293,  94.94949495,
        96.96969697,  98.98989899, 101.01010101, 103.03030303,
       105.05050505, 107.07070707, 109.09090909, 111.11111111,
       113.13131313, 115.15151515, 117.17171717, 119.19191919,
       121.21212121, 123.23232323, 125.25252525, 127.27

In [47]:
trajectories[0][1]

array([0.74055135, 0.7795419 ], dtype=float32)

In [54]:
# First 50 time points:
prey = trajectories[0, :1000, 0]
predator = trajectories[0, :50, 1]
times = time_points[:50]

trajectories.shape
prey.shape

(100,)

In [49]:
times

array([ 0.        ,  2.02020202,  4.04040404,  6.06060606,  8.08080808,
       10.1010101 , 12.12121212, 14.14141414, 16.16161616, 18.18181818,
       20.2020202 , 22.22222222, 24.24242424, 26.26262626, 28.28282828,
       30.3030303 , 32.32323232, 34.34343434, 36.36363636, 38.38383838,
       40.4040404 , 42.42424242, 44.44444444, 46.46464646, 48.48484848,
       50.50505051, 52.52525253, 54.54545455, 56.56565657, 58.58585859,
       60.60606061, 62.62626263, 64.64646465, 66.66666667, 68.68686869,
       70.70707071, 72.72727273, 74.74747475, 76.76767677, 78.78787879,
       80.80808081, 82.82828283, 84.84848485, 86.86868687, 88.88888889,
       90.90909091, 92.92929293, 94.94949495, 96.96969697, 98.98989899])

In [52]:
prey.shape

(100,)